<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Slave_Voyages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/slavevoyages/SlaveVoyages.png?raw=true" />

**Comércio Transatlântico de Escravos**

O site Slave Voyages, documentou e disponibilizou de forma digital, arquivos sobre esse triste período da história mundial e, em especial, para o Brasil, onde o tráfico negreiro perdurou até a criação da Lei Áurea de 1888. Existem registros de mercado de escravos, desde 1444 em Portugal. E existem registros bem antigos dessa prática, ainda em impérimos como o dos Romanos, Gregos, Árabes.  

### Sobre os dados

  * Explorar a disperssão de Africanos escravizados pelo Atlântico
    * https://www.slavevoyages.org/
    * Dataset: https://www.slavevoyages.org/voyage/database 

## Obtenção e transformação dos dados

**Carrega e verifica dataset**

In [1]:
# importando bibliotecas
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# importando dados
dataset = "https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/slavevoyages/data_SlaveVoyages.csv"
df_raw = pd.read_csv(dataset)

# entredas e variáveis
print('Entradas: {}' .format(df_raw.shape[0]))
print('Variáveis: {}' .format(df_raw.shape[1]))

Entradas: 36110
Variáveis: 8


In [3]:
# carregando dataset
df_raw.head()

,Identidade da viagem,Vessel name,Voyage itinerary imputed port where began (ptdepimp) place,Voyage itinerary imputed principal place of slave purchase (mjbyptimp),Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place,Year of arrival at port of disembarkation,Slaves arrived at 1st port,Captain's name
0,10354,Wakende Kraan,Hellevoetsluis,Elmina,Curacao,1706,177.0,"Hoeven, Jan van der<br/> Wenman, Roelof"
1,10355,Wapen van Amsterdam,NaN,Ardra,NaN,1685,NaN,NaN
2,10356,Wapen van Amsterdam,Texel,"Gold Coast, port unspecified",Curacao,1688,NaN,"Scheij, Pieter"
3,10357,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Curacao,1701,459.0,"Stoop, Pieter<br/> Crans, Pieter"
4,10358,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Suriname,1702,386.0,"Crans, Pieter"


In [4]:
# verificando os tipos de dados
df_raw.dtypes

Identidade da viagem                                                                   int64
Vessel name                                                                           object
Voyage itinerary imputed port where began (ptdepimp) place                            object
Voyage itinerary imputed principal place of slave purchase (mjbyptimp)                object
Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place     object
Year of arrival at port of disembarkation                                              int64
Slaves arrived at 1st port                                                           float64
Captain's name                                                                        object
dtype: object

In [5]:
# dados ausentes
df_raw.isnull().sum() / df_raw.shape[0] 

Identidade da viagem                                                                 0.000000
Vessel name                                                                          0.000083
Voyage itinerary imputed port where began (ptdepimp) place                           0.123705
Voyage itinerary imputed principal place of slave purchase (mjbyptimp)               0.046774
Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place    0.118056
Year of arrival at port of disembarkation                                            0.000000
Slaves arrived at 1st port                                                           0.491387
Captain's name                                                                       0.112323
dtype: float64

Em nosso dataset, identificamos que na métrica `Slaves arrived at 1st port`, que contém a quantidade de escravos, está com quase 50% dos dados ausentes. Isso dificulta nossa análise, uma vez que, nessa situação, será necessário excluir as entradas sem valor, que seria uma grande perda ou preencher com a média, médiana ou outros técnicas. 

**Transformação**

Para falicitar nas análises, vamos diminuir o tamanho do nome das columas e também, trazer para o português. Vamos criar um novo Dataset e manter o original, que servirá para consultas futuras.

In [7]:
# Dataset redução e tradução de colunas
df = df_raw.rename(columns={'Vessel name': 'Nome Embarcação', 
                            'Voyage itinerary imputed port where began (ptdepimp) place':'Porto onde viagem começou',
                            'Voyage itinerary imputed principal place of slave purchase (mjbyptimp) ':'Local de aquisição',
                            'Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place': 'Local Desembarque',
                            'Year of arrival at port of disembarkation': 'Ano Chegada',
                            'Slaves arrived at 1st port':'Qtde.',
                            "Captain's name":'Nome Capitão'})


# carregando dataset
df.head()

,Identidade da viagem,Nome Embarcação,Porto onde viagem começou,Local de aquisição,Local Desembarque,Ano Chegada,Qtde.,Nome Capitão
0,10354,Wakende Kraan,Hellevoetsluis,Elmina,Curacao,1706,177.0,"Hoeven, Jan van der<br/> Wenman, Roelof"
1,10355,Wapen van Amsterdam,NaN,Ardra,NaN,1685,NaN,NaN
2,10356,Wapen van Amsterdam,Texel,"Gold Coast, port unspecified",Curacao,1688,NaN,"Scheij, Pieter"
3,10357,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Curacao,1701,459.0,"Stoop, Pieter<br/> Crans, Pieter"
4,10358,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Suriname,1702,386.0,"Crans, Pieter"


Na exibição das 5 primeiras linhas, em algumas colunas `Local de aquisição`, `Nome Capitão`, conteúdo como: (', port unspecified') que polui a visuazliação e não contribui em nada. Vamos limpar. 

In [8]:
# retirando 'lixo' das entradas
df['Local Desembarque'] = df['Local Desembarque'].apply(lambda x: str(x).replace(', port unspecified', ''))
df['Local de aquisição'] = df['Local de aquisição'].apply(lambda x: str(x).replace(', port unspecified', ''))
df['Porto onde viagem começou'] = df['Porto onde viagem começou'].apply(lambda x: str(x).replace(', port unspecified', ''))

Para dados ausentes, vamos criar mais dois novos datasets e aplicar diferentes técnicas. No `df_clean`, vamos remover os ausentes. No `df_fill_median`, vamos preencher com a mediana. Assim, ao final, podemos comparar os resultados.

In [10]:
# criando dois novos datasets
df_clean = df.copy()
df_fill_median = df.copy()

In [11]:
# verificando dados ausentes
df.isnull().sum() / df.shape[0]

Identidade da viagem         0.000000
Nome Embarcação              0.000083
Porto onde viagem começou    0.000000
Local de aquisição           0.000000
Local Desembarque            0.000000
Ano Chegada                  0.000000
Qtde.                        0.491387
Nome Capitão                 0.112323
dtype: float64

In [14]:
# Excluir dados ausentes
df_clean = df_clean.dropna(subset=['Qtde.'], axis=0)

# exibi primeiras linhas
df_clean.isnull().sum()

Identidade da viagem            0
Nome Embarcação                 0
Porto onde viagem começou       0
Local de aquisição              0
Local Desembarque               0
Ano Chegada                     0
Qtde.                           0
Nome Capitão                 1626
dtype: int64

In [17]:
# preencher dados ausentes com a mediana
rd_median = df_fill_median['Qtde.'].median()
df_fill_median = df_fill_median.fillna({'Qtde.': rd_median})

# exibi primeiras linhas
df_fill_median.isnull().sum()

Identidade da viagem            0
Nome Embarcação                 3
Porto onde viagem começou       0
Local de aquisição              0
Local Desembarque               0
Ano Chegada                     0
Qtde.                           0
Nome Capitão                 4056
dtype: int64

In [20]:
# Verificando tipo de dados
df_fill_median.dtypes

Identidade da viagem           int64
Nome Embarcação               object
Porto onde viagem começou     object
Local de aquisição            object
Local Desembarque             object
Ano Chegada                    int64
Qtde.                        float64
Nome Capitão                  object
dtype: object

In [21]:
# converter tipo object to category
df_fill_median['Identidade da viagem'] = pd.Categorical(df_fill_median['Identidade da viagem'])
df_fill_median['Nome Embarcação'] = pd.Categorical(df_fill_median['Nome Embarcação'])
df_fill_median['Porto onde viagem começou'] = pd.Categorical(df_fill_median['Porto onde viagem começou'])
df_fill_median['Local de aquisição'] = pd.Categorical(df_fill_median['Local de aquisição'])
df_fill_median['Local Desembarque'] = pd.Categorical(df_fill_median['Local Desembarque'])
df_fill_median['Nome Capitão'] = pd.Categorical(df_fill_median['Nome Capitão'])
df_fill_median['Qtde.'] = df_fill_median['Qtde.'].astype('int64')

In [22]:
# verificando conversão
df_fill_median.dtypes

Identidade da viagem         category
Nome Embarcação              category
Porto onde viagem começou    category
Local de aquisição           category
Local Desembarque            category
Ano Chegada                     int64
Qtde.                           int64
Nome Capitão                 category
dtype: object

## Visualização

In [23]:
df_fill_median['Local Desembarque'].value_counts()[:10]

nan               4263
Bahia             4223
Rio de Janeiro    2887
Barbados          2038
Jamaica           1715
Kingston          1622
Pernambuco        1335
Havana            1322
Cap Francais      1127
Charleston         811
Name: Local Desembarque, dtype: int64

In [24]:
df_fill_median.head()

,Identidade da viagem,Nome Embarcação,Porto onde viagem começou,Local de aquisição,Local Desembarque,Ano Chegada,Qtde.,Nome Capitão
0,10354,Wakende Kraan,Hellevoetsluis,Elmina,Curacao,1706,177,"Hoeven, Jan van der<br/> Wenman, Roelof"
1,10355,Wapen van Amsterdam,nan,Ardra,nan,1685,253,NaN
2,10356,Wapen van Amsterdam,Texel,Gold Coast,Curacao,1688,253,"Scheij, Pieter"
3,10357,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Curacao,1701,459,"Stoop, Pieter<br/> Crans, Pieter"
4,10358,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Suriname,1702,386,"Crans, Pieter"


In [ ]:
fig, (ax, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax.bar(df['Ano Chegada'], df['Qtde.'])


plt.show()

In [ ]:
df.groupby(df['Ano Chegada']).sum()[:5].plot(kind='barh', figsize=(15, 2));

In [ ]:
df.columns

In [ ]:
df['Local Desembarque'].value_counts()[:10]

In [ ]:
df['Local Desembarque'] = df['Local Desembarque'].str.replace(', port unspecified', '').value_counts()[:10]

In [ ]:
df_teste = df.copy()

In [ ]:
df_teste.dtypes

In [ ]:
df_teste['Qtde.'] = df_teste['Qtde.'].replace(to_replace='NaN', value=1)

In [ ]:
df_teste_clean = df_teste.dropna(subset=['Qtde.'], axis=0)

In [ ]:
df_teste_clean.head()

In [ ]:
df_teste_clean['Nome Embarcação'] = pd.Categorical(df_teste_clean['Nome Embarcação'])

In [ ]:
df_teste_clean.dtypes

In [ ]:
df_teste_clean['Qtde.'] = df_teste_clean['Qtde.'].astype('int64')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(df_teste_clean['Local Desembarque'], df_teste_clean['Qtde.'])


plt.show()

In [ ]:
df_teste['Local Desembarque'] = df_teste['Local Desembarque'].apply(lambda x: str(x).replace(', port unspecified', ''))

In [ ]:
df_teste['Local Desembarque'].value_counts()[:10].plot(kind='barh');

In [ ]:
df_teste.head()

In [ ]:
df_teste['Local de aquisição'] = df_teste['Local de aquisição'].apply(lambda x: x.replace(', port unspecified', ''))

In [ ]:
df_teste['Local de aquisição'].value_counts()[:10].plot(kind='barh')

In [ ]:
df_teste['Qtde.'].apply(lambda x: int(x))

In [ ]:
df_teste['Qtde.'].dtypes

In [ ]:
fig, (ax, ax1) = plt.subplots(nrows=1, ncols=2)

ax.barh(df_teste['Local de aquisição'], df_teste['Qtde.'])


plt.show()

In [ ]:
df_teste.dtypes